<a href="https://colab.research.google.com/github/shubhiupa19/APResearch/blob/main/shubhi_ap_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all necessary libraries

In [ ]:
import pandas as pd
import io
import re
import nltk 
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split



Downloading all necessary nltk related data, such as stopwords etc for cleaning data later in the code

In [ ]:
nltk.download('wordnet')
nltk.download("stopwords")
nltk.download('punkt')

# stopwords = stopwords.words ("english")
from nltk.corpus import stopwords

# /* print (stopwords) */

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Reading the file from Github and create a Dataframe. Once it is downloaded, we save it in df_raw_data variable for future use

In [ ]:
#Utility function to remove stopwords

def removeStopWords( sentence):
   filtered_words = [word for word in sentence if word not in stopwords.words('english')]
   print( len(sentence) , len(filtered_words))

   return filtered_words


In [ ]:
#Get all of the data from private github repository

base_url_prefix = 'https://raw.githubusercontent.com/shubhiupa19/shubhi-ap-research/main/csv_files/'
file_numbers = range(300, 478)
url_suffix='_TRANSCRIPT.csv'

# df_raw_data = pd.DataFrame()

# for file_number in file_numbers:
#   try:
#     url = base_url_prefix + str(file_number) + url_suffix
#     print("url", url)
#     df = pd.read_csv(url, sep='\t', error_bad_lines=False)
#     df['Participant_ID'] = file_number
#     df_raw_data = pd.concat([df_raw_data, df])
#   except Exception as inst:
#     print(inst)
#     pass

# https://raw.githubusercontent.com/shubhiupa19/shubhi-ap-research/main/csv_files/300_TRANSCRIPT.csv
# https://raw.githubusercontent.com/shubhiupa19/shubhi-ap-research/main/csv_files/300_TRANSCRIPT.csv

In [ ]:
#display head of the raw data
df_raw_data.head()

NameError: ignored

Some data exploration to get a sense of it looks like

Download Training and Test data for future use in the program

In [ ]:
# Filter only those rows that where speaker is a Participant 

df_participant_only = df_raw_data[df_raw_data['speaker'] == 'Participant']

# and display only two columns
columns_to_display=['Participant_ID', 'value']
df_participant_only = df_participant_only [columns_to_display]

#print to see its values
print( "sample data", df_participant_only.head() )

NameError: ignored

In [ ]:
#Now Group all the values by a participant, so that all values for each Participant ID is concatenated in one long string

grouped_by_participant = df_participant_only.groupby('Participant_ID')[ 'value'].apply(list)
df_by_participant = grouped_by_participant.reset_index(name='value')

print("Sample data", df_by_participant[['Participant_ID', 'value']].head() )

NameError: ignored

In [ ]:
#Please dont run this again
df_train  = pd.read_csv("https://raw.githubusercontent.com/shubhiupa19/shubhi-ap-research/main/shubhi-train.csv")
df_test= pd.read_csv("https://raw.githubusercontent.com/shubhiupa19/shubhi-ap-research/main/shubhi-test-split.csv")

# These are the participants that have participated in training
train_participant_ids = df_train['Participant_ID']
print("total train_participant_ids  count >>",  len(train_participant_ids) )

# These participant IDs are kept for testing of models
test_participant_ids = df_test['Participant_ID']
print("test_participant_ids count >>",  len(test_participant_ids))

df_train.head()

total train_participant_ids  count >> 107
test_participant_ids count >> 47


,Participant_ID,PHQ8_Binary,PHQ8_Score,Gender,PHQ8_NoInterest,PHQ8_Depressed,PHQ8_Sleep,PHQ8_Tired,PHQ8_Appetite,PHQ8_Failure,PHQ8_Concentrating,PHQ8_Moving
0,303,0,0,0,0,0,0.0,0,0,0,0,0
1,304,0,6,0,0,1,1.0,2,2,0,0,0
2,305,0,7,1,0,1,1.0,2,2,1,0,0
3,310,0,4,1,1,1,0.0,0,0,1,1,0
4,312,0,2,1,0,0,1.0,1,0,0,0,0


Explore train and test data visually

Let us split the raw data responses based on the training and test data

In [ ]:
# TRAINING DATA 
#From raw data, Filter only those participant IDs that are part of the training set. r
#This would become our raw training data
df_for_training = df_by_participant[df_by_participant['Participant_ID'].isin( train_participant_ids ) ]
print(df_for_training.head())

In [ ]:
# TESTING DATA
#Similarly, From raw data, Filter only those participant IDs that are part of the testing set. r
#This would become our raw testing data
df_for_testing = df_by_participant[df_by_participant['Participant_ID'].isin( test_participant_ids ) ]
print(df_for_testing.head())

In [ ]:
# Create a countvectorizer for Training
#get the list of text responses from the training data. This would be the response text that each participant responded with
# for questions from interviewer.
participant_responses_values = df_for_training['value'].astype('U')

count_vect = CountVectorizer()

x_train_counts = count_vect.fit_transform(participant_responses_values)

tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)


In [ ]:
train_y = df_train['PHQ8_Score'][:100]


In [ ]:
# Now Create a Naive  Bayes Multinomial Model

# x_train_tfidf = x_train_tfidf.toarray()[0:100, 0:4389]
# train_x = x_train_tfidf[:70]

NBModel = MultinomialNB().fit(x_train_tfidf, train_y)
print("Model Details:", NBModel.get_params())


In [ ]:
# Create a countvectorizer for Testing
#get the list of text responses from the testing data. This would be the response text that each participant responded with
# for questions from interviewer.

testing_participant_ids = df_for_testing['Participant_ID']
test_raw_df = df_by_participant[df_by_participant['Participant_ID'].isin( testing_participant_ids ) ]
print(test_raw_df.head())

count_vect = CountVectorizer()
testing_participant_responses_list = test_raw_df['value'].astype('U')
x_test_counts = count_vect.fit_transform(testing_participant_responses_list)

tfidf_transformer = TfidfTransformer()
x_test_tfidf = tfidf_transformer.fit_transform(x_test_counts)


NameError: ignored

In [ ]:
# Now predict the PHQ8 Score of testing participants 
# and compare the values f

y_score = NBModel.predict(x_test_tfidf)
print(df_test.head())
test_y = df_test['PHQ_Score']

n_right = 0
for i in range(len(y_score)):
    print("y_score[i]: ", y_score[i], "test_y[i] : ", test_y[i])
    
    if y_score[i] == test_y[i]:
       n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))


NameError: ignored